In [1]:
import pandas as pd
import json

from datetime import datetime

In [2]:
def display_fanta_squad(predictions, teams, index):
    fanta_team = predictions[predictions['fanta_team'] == teams[index]]
    return fanta_team, index+1    

In [3]:
try:
    
    ###############################################################################################
    status = 'Loading config ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    with open('config/config.json') as config:
        config = json.load(config)
    status = 'Loading config done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Opening predictions ...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    filename = '{}_{}_{}_{}'.format(config['league_name'], pd.Timestamp('today').day, pd.Timestamp('today').month, pd.Timestamp('today').year)
    predictions = pd.read_csv('data/outputs/predictions/{}.csv'.format(filename))
    status = 'Opening predictions done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

    ###############################################################################################
    status = 'Preparing predictions for display...'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    predictions['player'] = predictions['player'].apply(lambda x: x.split(' ')[-1])
    status = 'Preparing predictions for display done!'
    print('{} -> {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), status))
    ###############################################################################################

except Exception as e:
    print('Exception: {}'.format(e))
    print('Errored out during: {}'.format(status))

2022-01-22 22:20:44 -> Loading config ...
2022-01-22 22:20:44 -> Loading config done!
2022-01-22 22:20:44 -> Opening predictions ...
2022-01-22 22:20:44 -> Opening predictions done!
2022-01-22 22:20:44 -> Preparing predictions for display...
2022-01-22 22:20:44 -> Preparing predictions for display done!


In [4]:
_deepnote_run_altair(predictions, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false},"aggregate":"sum"},"y":{"field":"fanta_team","type":"nominal","sort":{"order":"descending","encoding":"x"},"scale":{"type":"linear","zero":true}},"color":{"field":"fanta_team","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [5]:
teams = sorted(list(predictions['fanta_team'].unique()))
index = 0

In [6]:
team, index = display_fanta_squad(predictions, teams, index)

In [7]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [8]:
team, index = display_fanta_squad(predictions, teams, index)

In [9]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [10]:
team, index = display_fanta_squad(predictions, teams, index)

In [11]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [12]:
team, index = display_fanta_squad(predictions, teams, index)

In [13]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [14]:
team, index = display_fanta_squad(predictions, teams, index)

In [15]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [16]:
team, index = display_fanta_squad(predictions, teams, index)

In [17]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [18]:
team, index = display_fanta_squad(predictions, teams, index)

In [19]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [20]:
team, index = display_fanta_squad(predictions, teams, index)

In [21]:
_deepnote_run_altair(team, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"expected_bonus","type":"quantitative","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"player","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"role","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82f6a53d-2812-4762-aa0b-0b8fac2a34e3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>